In [20]:
import duckdb
import pandas as pd

%load_ext sql
con = duckdb.connect("../../data/finance_data.db")
%sql con --alias duckdb
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [21]:
%%sql df <<

with scores as (
    select 
        * 
    from afv_20_scores 
    order by afv desc 
),
    
infos as (
    select
        symbol,
        cast(json_value(data, '$.marketCap.0') as bigint) as market_cap,
        json_extract_string(data, '$.country.0') as country,
    json_extract_string(data, '$.industry.0') as industry,
        json_extract_string(data, '$.averageDailyVolume3Month.0') as avg_daily_volume
    from yahoo_data
    where dataset = 'info'
)

select
    scores.*,
    infos.market_cap / 1e9 as market_cap_billion,
    infos.industry,
    infos.country,
    infos.avg_daily_volume
from scores
left join infos
    on infos.symbol = scores.symbol
limit 500

In [22]:
from datetime import datetime

date_str = datetime.today().strftime("%Y-%m-%d")
filename = f"afv_results_top50_{date_str}.csv"
df.to_csv(filename, index=False)

In [23]:
con.close()